# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 20 2023 11:17AM,261460,15,PNC442792A,"Person & Covey, Inc.",Released
1,Apr 20 2023 11:17AM,261482,15,PNC442827B,"Person & Covey, Inc.",Released
2,Apr 20 2023 11:16AM,261483,15,9387621,"Brookfield Pharmaceuticals, LLC",Released
3,Apr 20 2023 11:16AM,261500,21,854366,"NBTY Global, Inc.",Released
4,Apr 20 2023 11:07AM,261499,10,Enova-11691,Emerginnova,Released
5,Apr 20 2023 11:07AM,261499,10,Enova-11692,Emerginnova,Released
6,Apr 20 2023 10:55AM,261498,10,MSP220966,"Methapharm, Inc.",Released
7,Apr 20 2023 10:55AM,261498,10,MSP220969,"Methapharm, Inc.",Released
8,Apr 20 2023 10:55AM,261498,10,MSP220970,"Methapharm, Inc.",Released
9,Apr 20 2023 10:55AM,261498,10,MSP220971,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,261495,Released,1
30,261496,Released,1
31,261498,Released,12
32,261499,Released,2
33,261500,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
261495,NaN,1.0
261496,NaN,1.0
261498,NaN,12.0
261499,NaN,2.0
261500,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
261400,22.0,20.0
261432,0.0,1.0
261448,0.0,5.0
261452,0.0,1.0
261460,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
261400,22,20
261432,0,1
261448,0,5
261452,0,1
261460,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,261400,22,20
1,261432,0,1
2,261448,0,5
3,261452,0,1
4,261460,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,261400,22,20
1,261432,,1
2,261448,,5
3,261452,,1
4,261460,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 20 2023 11:17AM,261460,15,"Person & Covey, Inc."
1,Apr 20 2023 11:17AM,261482,15,"Person & Covey, Inc."
2,Apr 20 2023 11:16AM,261483,15,"Brookfield Pharmaceuticals, LLC"
3,Apr 20 2023 11:16AM,261500,21,"NBTY Global, Inc."
4,Apr 20 2023 11:07AM,261499,10,Emerginnova
6,Apr 20 2023 10:55AM,261498,10,"Methapharm, Inc."
18,Apr 20 2023 10:52AM,261496,10,"Methapharm, Inc."
19,Apr 20 2023 10:43AM,261495,10,"Senseonics, Incorporated"
20,Apr 20 2023 10:41AM,261494,10,"ClearSpec, LLC"
21,Apr 20 2023 10:37AM,261493,19,"Graystone, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Apr 20 2023 11:17AM,261460,15,"Person & Covey, Inc.",,1
1,Apr 20 2023 11:17AM,261482,15,"Person & Covey, Inc.",,1
2,Apr 20 2023 11:16AM,261483,15,"Brookfield Pharmaceuticals, LLC",,1
3,Apr 20 2023 11:16AM,261500,21,"NBTY Global, Inc.",,1
4,Apr 20 2023 11:07AM,261499,10,Emerginnova,,2
5,Apr 20 2023 10:55AM,261498,10,"Methapharm, Inc.",,12
6,Apr 20 2023 10:52AM,261496,10,"Methapharm, Inc.",,1
7,Apr 20 2023 10:43AM,261495,10,"Senseonics, Incorporated",,1
8,Apr 20 2023 10:41AM,261494,10,"ClearSpec, LLC",,1
9,Apr 20 2023 10:37AM,261493,19,"Graystone, LLC",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 20 2023 11:17AM,261460,15,"Person & Covey, Inc.",1,
1,Apr 20 2023 11:17AM,261482,15,"Person & Covey, Inc.",1,
2,Apr 20 2023 11:16AM,261483,15,"Brookfield Pharmaceuticals, LLC",1,
3,Apr 20 2023 11:16AM,261500,21,"NBTY Global, Inc.",1,
4,Apr 20 2023 11:07AM,261499,10,Emerginnova,2,
5,Apr 20 2023 10:55AM,261498,10,"Methapharm, Inc.",12,
6,Apr 20 2023 10:52AM,261496,10,"Methapharm, Inc.",1,
7,Apr 20 2023 10:43AM,261495,10,"Senseonics, Incorporated",1,
8,Apr 20 2023 10:41AM,261494,10,"ClearSpec, LLC",1,
9,Apr 20 2023 10:37AM,261493,19,"Graystone, LLC",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 20 2023 11:17AM,261460,15,"Person & Covey, Inc.",1,
1,Apr 20 2023 11:17AM,261482,15,"Person & Covey, Inc.",1,
2,Apr 20 2023 11:16AM,261483,15,"Brookfield Pharmaceuticals, LLC",1,
3,Apr 20 2023 11:16AM,261500,21,"NBTY Global, Inc.",1,
4,Apr 20 2023 11:07AM,261499,10,Emerginnova,2,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Apr 20 2023 11:17AM,261460,15,"Person & Covey, Inc.",1.0,NaN
1,Apr 20 2023 11:17AM,261482,15,"Person & Covey, Inc.",1.0,NaN
2,Apr 20 2023 11:16AM,261483,15,"Brookfield Pharmaceuticals, LLC",1.0,NaN
3,Apr 20 2023 11:16AM,261500,21,"NBTY Global, Inc.",1.0,NaN
4,Apr 20 2023 11:07AM,261499,10,Emerginnova,2.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 20 2023 11:17AM,261460,15,"Person & Covey, Inc.",1.0,0.0
1,Apr 20 2023 11:17AM,261482,15,"Person & Covey, Inc.",1.0,0.0
2,Apr 20 2023 11:16AM,261483,15,"Brookfield Pharmaceuticals, LLC",1.0,0.0
3,Apr 20 2023 11:16AM,261500,21,"NBTY Global, Inc.",1.0,0.0
4,Apr 20 2023 11:07AM,261499,10,Emerginnova,2.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2353410,29.0,16.0
15,1307273,28.0,22.0
19,3137654,89.0,16.0
21,522986,1.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2353410,29.0,16.0
1,15,1307273,28.0,22.0
2,19,3137654,89.0,16.0
3,21,522986,1.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,29.0,16.0
1,15,28.0,22.0
2,19,89.0,16.0
3,21,1.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,29.0
1,15,Released,28.0
2,19,Released,89.0
3,21,Released,1.0
4,10,Executing,16.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,21
Status,,,,
Executing,16.0,22.0,16.0,1.0
Released,29.0,28.0,89.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,21
0,Executing,16.0,22.0,16.0,1.0
1,Released,29.0,28.0,89.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,21
0,Executing,16.0,22.0,16.0,1.0
1,Released,29.0,28.0,89.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()